In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os
from PIL import Image
import math
import time
%matplotlib inline

In [7]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.utils import np_utils
from keras import regularizers
from keras.callbacks import EarlyStopping
import keras 
import h5py
model = load_model('conv_adam_lr0.0001_batch128_epoch20.h5')

In [8]:
def map_emotion(prediction):
    highest = max(prediction)
    emotion = []
    for pred in prediction:
        if pred == highest:
            emotion.append(1)
        else:
            emotion.append(0)
    emotions = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    for a, b in zip(emotion, emotions):
        if a == 1:
            return b

In [9]:
haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
font = cv2.FONT_HERSHEY_SIMPLEX

In [10]:
def takePhoto():
    video_capture = cv2.VideoCapture(0)
    video_capture.set(3,1280)
    video_capture.set(4,960)
    predicts = []
    while True:
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = haar_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=8,
            minSize=(100, 100),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        cv2.putText(frame,'Emotion:',(0,50), font, 1.5,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(frame,'Anger',(0,100), font, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(frame,'Disgust',(0,150), font, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(frame,'Fear',(0,200), font, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(frame,'Happiness',(0,250), font, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(frame,'Sadness',(0,300), font, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(frame,'Surprise',(0,350), font, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(frame,'Neutrality',(0,400), font, 1,(255,255,255),2,cv2.LINE_AA)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            gray_face = cv2.resize(gray[y:(y+h), x:(x+w)], (48, 48), interpolation = cv2.INTER_CUBIC)
            gray_face = gray_face.reshape([-1, 48, 48, 1])
            prediction = model.predict(gray_face)
            percentages = [(str(np.around(prediction[0][i] * 100)) + '%') for i in range(7)]
            cv2.putText(frame, percentages[0],(200,100), font, 1,(0,255,255),2,cv2.LINE_AA)
            cv2.putText(frame, percentages[1],(200,150), font, 1,(0,255,255),2,cv2.LINE_AA)
            cv2.putText(frame, percentages[2],(200,200), font, 1,(0,255,255),2,cv2.LINE_AA)
            cv2.putText(frame, percentages[3],(200,250), font, 1,(0,255,255),2,cv2.LINE_AA)
            cv2.putText(frame, percentages[4],(200,300), font, 1,(0,255,255),2,cv2.LINE_AA)
            cv2.putText(frame, percentages[5],(200,350), font, 1,(0,255,255),2,cv2.LINE_AA)
            cv2.putText(frame, percentages[6],(200,400), font, 1,(0,255,255),2,cv2.LINE_AA)
            cv2.putText(frame, map_emotion(prediction[0]), (250, 50), font, 1,(0,255,255),2,cv2.LINE_AA)
            a = 0
            for i in range(30000):
                a = i
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_capture.release()
    cv2.destroyAllWindows()

In [11]:
takePhoto()